In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50)
from scipy.spatial import cKDTree
from math import *
np.random.seed(123)
import urllib.request
import urllib, os

In [2]:
import pandas as pd
import xarray as xr
import math
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta

In [3]:
import numpy as np
import xarray as xr
import xesmf as xe

In [4]:
df_prep = xr.open_dataset(r'/home/shubham/ugp/cl_new/Grace_2002-2021.nc')

In [5]:
df_prep

<xarray.Dataset>
Dimensions:        (time: 236, y: 180, x: 360)
Coordinates:
  * time           (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01
  * x              (x) int64 0 1 2 3 4 5 6 7 ... 352 353 354 355 356 357 358 359
  * y              (y) float64 -90.0 -89.0 -88.0 -87.0 ... 86.0 87.0 88.0 89.0
Data variables:
    lwe_thickness  (time, y, x) float64 ...

In [6]:
df_prep["slice"] = df_prep["lwe_thickness"].isel(time=0)
df_prep

<xarray.Dataset>
Dimensions:        (time: 236, y: 180, x: 360)
Coordinates:
  * time           (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01
  * x              (x) int64 0 1 2 3 4 5 6 7 ... 352 353 354 355 356 357 358 359
  * y              (y) float64 -90.0 -89.0 -88.0 -87.0 ... 86.0 87.0 88.0 89.0
Data variables:
    lwe_thickness  (time, y, x) float64 ...
    slice          (y, x) float64 ...

In [7]:
df_prep = df_prep.rename({'x':'lon', 'y':'lat'})

In [8]:
df_prep

<xarray.Dataset>
Dimensions:        (time: 236, lat: 180, lon: 360)
Coordinates:
  * time           (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01
  * lon            (lon) int64 0 1 2 3 4 5 6 7 ... 353 354 355 356 357 358 359
  * lat            (lat) float64 -90.0 -89.0 -88.0 -87.0 ... 86.0 87.0 88.0 89.0
Data variables:
    lwe_thickness  (time, lat, lon) float64 ...
    slice          (lat, lon) float64 0.0 0.0 0.0 0.0 ... -1.398 -1.398 -1.398

In [9]:
df_prep_out = xr.Dataset(
    {
        "lat": (["lat"], np.arange(-90,90, 2.0)),
        "lon": (["lon"], np.arange(0, 360, 2)),
    }
)

regridder = xe.Regridder(df_prep, df_prep_out, 'bilinear')
regridder

Create weight file: bilinear_180x360_90x180.nc


xESMF Regridder 
Regridding algorithm:       bilinear 
Weight filename:            bilinear_180x360_90x180.nc 
Reuse pre-computed weights? False 
Input grid shape:           (180, 360) 
Output grid shape:          (90, 180) 
Output grid dimension name: ('lat', 'lon') 
Periodic in longitude?      False

In [10]:
df_out = regridder(df_prep)
df_out

using dimensions ('lat', 'lon') from data variable lwe_thickness as the horizontal dimensions for this dataset.


/home/shubham/anaconda3/envs/ugp_env/lib/python3.7/site-packages/xesmf/frontend.py:486: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs


<xarray.Dataset>
Dimensions:        (time: 236, lat: 90, lon: 180)
Coordinates:
  * time           (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01
  * lon            (lon) int64 0 2 4 6 8 10 12 ... 346 348 350 352 354 356 358
  * lat            (lat) float64 -90.0 -88.0 -86.0 -84.0 ... 82.0 84.0 86.0 88.0
Data variables:
    lwe_thickness  (time, lat, lon) float64 0.0 0.0 0.0 ... 1.194 1.194 1.194
    slice          (lat, lon) float64 0.0 0.0 0.0 0.0 ... -1.436 -1.436 -1.436
Attributes:
    regrid_method:  bilinear

In [11]:
df_out = df_out.drop(['slice'])

In [12]:
df_out['time'][:]

<xarray.DataArray 'time' (time: 236)>
array(['2002-04-01T00:00:00.000000000', '2002-05-01T00:00:00.000000000',
       '2002-06-01T00:00:00.000000000', ..., '2021-09-01T00:00:00.000000000',
       '2021-10-01T00:00:00.000000000', '2021-11-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01

In [13]:
df_out = df_out['lwe_thickness'][:][:][:]

In [14]:
df_out

<xarray.DataArray 'lwe_thickness' (time: 236, lat: 90, lon: 180)>
array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        , -4.55867815, -4.55867815, ..., -4.55867815,
         -4.55867815, -4.55867815],
        [ 0.        , -5.78173685, -5.78173685, ..., -5.78173685,
         -5.78173685, -5.78173685],
        ...,
        [ 0.        ,  3.29571748,  3.29571748, ...,  3.77121197,
          3.29571748,  3.29571748],
        [ 0.        , -0.07990051, -0.07990051, ..., -0.07990051,
         -0.07990051, -0.07990051],
        [ 0.        , -1.43616831, -1.43616831, ..., -1.43616831,
         -1.43616831, -1.43616831]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        , -3.52208829, -3.52208829, ..., -3.52208829,
         -3.52208829, -3.52208829],
        [ 0.        , -4.56921482, -4.56921482, ..., -4.56921482,
         -4.56921482, -4.56921482],
...
        [ 0.        , -2.22687268, -2.22687268, ..., -5.17118675,
         -2.22687268, -2.22687268],
        [ 0.        ,  3.41264534,  3.41264534, ...,  3.41264534,
          3.41264534,  3.41264534],
        [ 0.        ,  0.17298652,  0.17298652, ...,  0.17298652,
          0.17298652,  0.17298652]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  9.25846863,  9.25846863, ...,  9.25846863,
          9.25846863,  9.25846863],
        [ 0.        , 10.56460476, 10.56460476, ..., 10.56460476,
         10.56460476, 10.56460476],
        ...,
        [ 0.        , -4.26758337, -4.26758337, ..., -6.22503031,
         -4.26758337, -4.26758337],
        [ 0.        ,  2.69238806,  2.69238806, ...,  2.69238806,
          2.69238806,  2.69238806],
        [ 0.        ,  1.19432092,  1.19432092, ...,  1.19432092,
          1.19432092,  1.19432092]]])
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01
  * lon      (lon) int64 0 2 4 6 8 10 12 14 ... 344 346 348 350 352 354 356 358
  * lat      (lat) float64 -90.0 -88.0 -86.0 -84.0 -82.0 ... 82.0 84.0 86.0 88.0

In [15]:
pd.to_datetime(df_out['time'])[0]

Timestamp('2002-04-01 00:00:00')

In [16]:
new_index = pd.period_range(pd.to_datetime(df_out['time'])[0],pd.to_datetime(df_out['time'])[-1],freq='M')

In [17]:
new_index = pd.DataFrame(new_index)
new_index

,0
0,2002-04
1,2002-05
2,2002-06
3,2002-07
4,2002-08
...,...
231,2021-07
232,2021-08
233,2021-09
234,2021-10


In [18]:
df_out

<xarray.DataArray 'lwe_thickness' (time: 236, lat: 90, lon: 180)>
array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        , -4.55867815, -4.55867815, ..., -4.55867815,
         -4.55867815, -4.55867815],
        [ 0.        , -5.78173685, -5.78173685, ..., -5.78173685,
         -5.78173685, -5.78173685],
        ...,
        [ 0.        ,  3.29571748,  3.29571748, ...,  3.77121197,
          3.29571748,  3.29571748],
        [ 0.        , -0.07990051, -0.07990051, ..., -0.07990051,
         -0.07990051, -0.07990051],
        [ 0.        , -1.43616831, -1.43616831, ..., -1.43616831,
         -1.43616831, -1.43616831]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        , -3.52208829, -3.52208829, ..., -3.52208829,
         -3.52208829, -3.52208829],
        [ 0.        , -4.56921482, -4.56921482, ..., -4.56921482,
         -4.56921482, -4.56921482],
...
        [ 0.        , -2.22687268, -2.22687268, ..., -5.17118675,
         -2.22687268, -2.22687268],
        [ 0.        ,  3.41264534,  3.41264534, ...,  3.41264534,
          3.41264534,  3.41264534],
        [ 0.        ,  0.17298652,  0.17298652, ...,  0.17298652,
          0.17298652,  0.17298652]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  9.25846863,  9.25846863, ...,  9.25846863,
          9.25846863,  9.25846863],
        [ 0.        , 10.56460476, 10.56460476, ..., 10.56460476,
         10.56460476, 10.56460476],
        ...,
        [ 0.        , -4.26758337, -4.26758337, ..., -6.22503031,
         -4.26758337, -4.26758337],
        [ 0.        ,  2.69238806,  2.69238806, ...,  2.69238806,
          2.69238806,  2.69238806],
        [ 0.        ,  1.19432092,  1.19432092, ...,  1.19432092,
          1.19432092,  1.19432092]]])
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01
  * lon      (lon) int64 0 2 4 6 8 10 12 14 ... 344 346 348 350 352 354 356 358
  * lat      (lat) float64 -90.0 -88.0 -86.0 -84.0 -82.0 ... 82.0 84.0 86.0 88.0

In [19]:
type(new_index[0][0])

pandas._libs.tslibs.period.Period

In [20]:
new_index = new_index[0].dt.to_timestamp()
type(new_index)

pandas.core.series.Series

In [21]:
new_index

0     2002-04-01
1     2002-05-01
2     2002-06-01
3     2002-07-01
4     2002-08-01
         ...    
231   2021-07-01
232   2021-08-01
233   2021-09-01
234   2021-10-01
235   2021-11-01
Name: 0, Length: 236, dtype: datetime64[ns]

In [22]:
df_out['time'] = pd.to_datetime(df_out['time']).strftime("%Y-%m")

In [23]:
df_out['time'] = pd.to_datetime(df_out['time'])
df_out

<xarray.DataArray 'lwe_thickness' (time: 236, lat: 90, lon: 180)>
array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        , -4.55867815, -4.55867815, ..., -4.55867815,
         -4.55867815, -4.55867815],
        [ 0.        , -5.78173685, -5.78173685, ..., -5.78173685,
         -5.78173685, -5.78173685],
        ...,
        [ 0.        ,  3.29571748,  3.29571748, ...,  3.77121197,
          3.29571748,  3.29571748],
        [ 0.        , -0.07990051, -0.07990051, ..., -0.07990051,
         -0.07990051, -0.07990051],
        [ 0.        , -1.43616831, -1.43616831, ..., -1.43616831,
         -1.43616831, -1.43616831]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        , -3.52208829, -3.52208829, ..., -3.52208829,
         -3.52208829, -3.52208829],
        [ 0.        , -4.56921482, -4.56921482, ..., -4.56921482,
         -4.56921482, -4.56921482],
...
        [ 0.        , -2.22687268, -2.22687268, ..., -5.17118675,
         -2.22687268, -2.22687268],
        [ 0.        ,  3.41264534,  3.41264534, ...,  3.41264534,
          3.41264534,  3.41264534],
        [ 0.        ,  0.17298652,  0.17298652, ...,  0.17298652,
          0.17298652,  0.17298652]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  9.25846863,  9.25846863, ...,  9.25846863,
          9.25846863,  9.25846863],
        [ 0.        , 10.56460476, 10.56460476, ..., 10.56460476,
         10.56460476, 10.56460476],
        ...,
        [ 0.        , -4.26758337, -4.26758337, ..., -6.22503031,
         -4.26758337, -4.26758337],
        [ 0.        ,  2.69238806,  2.69238806, ...,  2.69238806,
          2.69238806,  2.69238806],
        [ 0.        ,  1.19432092,  1.19432092, ...,  1.19432092,
          1.19432092,  1.19432092]]])
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01
  * lon      (lon) int64 0 2 4 6 8 10 12 14 ... 344 346 348 350 352 354 356 358
  * lat      (lat) float64 -90.0 -88.0 -86.0 -84.0 -82.0 ... 82.0 84.0 86.0 88.0

In [24]:
df_out['time'][:]

<xarray.DataArray 'time' (time: 236)>
array(['2002-04-01T00:00:00.000000000', '2002-05-01T00:00:00.000000000',
       '2002-06-01T00:00:00.000000000', ..., '2021-09-01T00:00:00.000000000',
       '2021-10-01T00:00:00.000000000', '2021-11-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01

In [25]:
new_index

0     2002-04-01
1     2002-05-01
2     2002-06-01
3     2002-07-01
4     2002-08-01
         ...    
231   2021-07-01
232   2021-08-01
233   2021-09-01
234   2021-10-01
235   2021-11-01
Name: 0, Length: 236, dtype: datetime64[ns]

In [26]:
df_out['lat']

<xarray.DataArray 'lat' (lat: 90)>
array([-90., -88., -86., -84., -82., -80., -78., -76., -74., -72., -70., -68.,
       -66., -64., -62., -60., -58., -56., -54., -52., -50., -48., -46., -44.,
       -42., -40., -38., -36., -34., -32., -30., -28., -26., -24., -22., -20.,
       -18., -16., -14., -12., -10.,  -8.,  -6.,  -4.,  -2.,   0.,   2.,   4.,
         6.,   8.,  10.,  12.,  14.,  16.,  18.,  20.,  22.,  24.,  26.,  28.,
        30.,  32.,  34.,  36.,  38.,  40.,  42.,  44.,  46.,  48.,  50.,  52.,
        54.,  56.,  58.,  60.,  62.,  64.,  66.,  68.,  70.,  72.,  74.,  76.,
        78.,  80.,  82.,  84.,  86.,  88.])
Coordinates:
  * lat      (lat) float64 -90.0 -88.0 -86.0 -84.0 -82.0 ... 82.0 84.0 86.0 88.0

In [27]:
df_out

<xarray.DataArray 'lwe_thickness' (time: 236, lat: 90, lon: 180)>
array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        , -4.55867815, -4.55867815, ..., -4.55867815,
         -4.55867815, -4.55867815],
        [ 0.        , -5.78173685, -5.78173685, ..., -5.78173685,
         -5.78173685, -5.78173685],
        ...,
        [ 0.        ,  3.29571748,  3.29571748, ...,  3.77121197,
          3.29571748,  3.29571748],
        [ 0.        , -0.07990051, -0.07990051, ..., -0.07990051,
         -0.07990051, -0.07990051],
        [ 0.        , -1.43616831, -1.43616831, ..., -1.43616831,
         -1.43616831, -1.43616831]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        , -3.52208829, -3.52208829, ..., -3.52208829,
         -3.52208829, -3.52208829],
        [ 0.        , -4.56921482, -4.56921482, ..., -4.56921482,
         -4.56921482, -4.56921482],
...
        [ 0.        , -2.22687268, -2.22687268, ..., -5.17118675,
         -2.22687268, -2.22687268],
        [ 0.        ,  3.41264534,  3.41264534, ...,  3.41264534,
          3.41264534,  3.41264534],
        [ 0.        ,  0.17298652,  0.17298652, ...,  0.17298652,
          0.17298652,  0.17298652]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  9.25846863,  9.25846863, ...,  9.25846863,
          9.25846863,  9.25846863],
        [ 0.        , 10.56460476, 10.56460476, ..., 10.56460476,
         10.56460476, 10.56460476],
        ...,
        [ 0.        , -4.26758337, -4.26758337, ..., -6.22503031,
         -4.26758337, -4.26758337],
        [ 0.        ,  2.69238806,  2.69238806, ...,  2.69238806,
          2.69238806,  2.69238806],
        [ 0.        ,  1.19432092,  1.19432092, ...,  1.19432092,
          1.19432092,  1.19432092]]])
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01
  * lon      (lon) int64 0 2 4 6 8 10 12 14 ... 344 346 348 350 352 354 356 358
  * lat      (lat) float64 -90.0 -88.0 -86.0 -84.0 -82.0 ... 82.0 84.0 86.0 88.0

In [28]:
x = df_out.reindex({"time": new_index})
x

<xarray.DataArray 'lwe_thickness' (time: 236, lat: 90, lon: 180)>
array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        , -4.55867815, -4.55867815, ..., -4.55867815,
         -4.55867815, -4.55867815],
        [ 0.        , -5.78173685, -5.78173685, ..., -5.78173685,
         -5.78173685, -5.78173685],
        ...,
        [ 0.        ,  3.29571748,  3.29571748, ...,  3.77121197,
          3.29571748,  3.29571748],
        [ 0.        , -0.07990051, -0.07990051, ..., -0.07990051,
         -0.07990051, -0.07990051],
        [ 0.        , -1.43616831, -1.43616831, ..., -1.43616831,
         -1.43616831, -1.43616831]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        , -3.52208829, -3.52208829, ..., -3.52208829,
         -3.52208829, -3.52208829],
        [ 0.        , -4.56921482, -4.56921482, ..., -4.56921482,
         -4.56921482, -4.56921482],
...
        [ 0.        , -2.22687268, -2.22687268, ..., -5.17118675,
         -2.22687268, -2.22687268],
        [ 0.        ,  3.41264534,  3.41264534, ...,  3.41264534,
          3.41264534,  3.41264534],
        [ 0.        ,  0.17298652,  0.17298652, ...,  0.17298652,
          0.17298652,  0.17298652]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  9.25846863,  9.25846863, ...,  9.25846863,
          9.25846863,  9.25846863],
        [ 0.        , 10.56460476, 10.56460476, ..., 10.56460476,
         10.56460476, 10.56460476],
        ...,
        [ 0.        , -4.26758337, -4.26758337, ..., -6.22503031,
         -4.26758337, -4.26758337],
        [ 0.        ,  2.69238806,  2.69238806, ...,  2.69238806,
          2.69238806,  2.69238806],
        [ 0.        ,  1.19432092,  1.19432092, ...,  1.19432092,
          1.19432092,  1.19432092]]])
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01
  * lon      (lon) int64 0 2 4 6 8 10 12 14 ... 344 346 348 350 352 354 356 358
  * lat      (lat) float64 -90.0 -88.0 -86.0 -84.0 -82.0 ... 82.0 84.0 86.0 88.0

In [30]:
x.to_netcdf('/home/shubham/ugp/cl_new/regrid_2_deg/grace_04-2002_11-2021.nc')

In [31]:
df_mean = x.mean()

In [32]:
df_mean

<xarray.DataArray 'lwe_thickness' ()>
array(-0.75225288)